In [4]:
import pandas as pd
import numpy as np
from pathlib import Path

In [68]:
path_log = '/mnt/HC_Volume_18315164/home-jupyter/jupyter-p-syrnev/shared/homeworks/python_ds_miniprojects/5_subsid/subsid/prod_activations_logs.csv'

In [85]:
log_df = pd.read_csv(path_log, parse_dates=['START_DTTM','END_DTTM'], dayfirst=[True, True], sep=';', index_col=0)

In [89]:
log_df = log_df.reset_index(drop=True)
log_df['difference'] = log_df.END_DTTM - log_df.START_DTTM

log_df

,SUBS_ID,PROD_ID,START_DTTM,END_DTTM,difference
0,id4651830,1954,2020-03-20 14:59:00,2020-12-01 00:00:00,255 days 09:01:00
1,id7646509,6431,2020-03-19 13:00:00,2020-03-19 13:03:00,0 days 00:03:00
2,id7461794,3310,2020-03-20 17:25:00,2020-12-01 00:00:00,255 days 06:35:00
3,id5416547,1743,2020-03-17 10:17:00,2020-03-25 11:00:00,8 days 00:43:00
4,id8238421,1859,2020-03-01 11:42:00,2020-03-01 11:43:00,0 days 00:01:00
5,id8641743,2752,2020-03-20 15:44:00,2020-04-21 15:44:00,32 days 00:00:00
6,id2185490,3210,2020-03-16 16:28:00,2020-12-01 00:00:00,259 days 07:32:00
7,id7642700,3020,2020-03-15 14:21:00,2020-03-15 23:42:00,0 days 09:21:00
8,id8741631,5677,2020-03-19 12:28:00,2020-12-01 00:00:00,256 days 11:32:00


In [92]:
#log_df.difference > '00:05:00'
#log_df.difference > '5m'

In [94]:
log_df_success = log_df.query('difference > "00:05:00"')

In [52]:
root = Path('/mnt/HC_Volume_18315164/home-jupyter/jupyter-p-syrnev/shared/homeworks/python_ds_miniprojects/5_subsid/subsid/')

tm_sales_df = pd.DataFrame()
for path in root.rglob('tm_sales_*.csv'):
    tmp_df = pd.read_csv(path, sep=';', parse_dates=['ACT_DTTM'], dayfirst=True)
    tm_sales_df = pd.concat([tm_sales_df, tmp_df])

tm_sales_df = tm_sales_df.dropna(subset=['SUBS_ID']).reset_index(drop=True)

tm_sales_df.SUBS_ID = tm_sales_df.SUBS_ID.apply(lambda x: x if 'id' in x else 'id' + x)

In [62]:
tm_sales_df

,SUBS_ID,FILIAL_ID,PROD_ID,ACT_DTTM
0,id4651830,1,1954,2020-03-20 14:59:00
1,id7646509,5,6431,2020-03-19 13:00:00
2,id7412683,4,3313,2020-03-22 17:25:00
3,id5416547,3,1743,2020-03-17 10:17:00
4,id8362218,7,9879,2020-03-05 11:42:00
5,id2185490,2,3210,2020-03-16 16:28:00
6,id5764122,3,1499,2020-03-18 15:44:00
7,id7642700,6,3020,2020-03-15 14:21:00
8,id1374509,2,5677,2020-03-17 11:48:00


In [98]:
interested_columns = ['SUBS_ID', 'FILIAL_ID', 'ACT_DTTM']
success = log_df_success.merge(tm_sales_df[interested_columns], on='SUBS_ID')
success

,SUBS_ID,PROD_ID,START_DTTM,END_DTTM,difference,FILIAL_ID,ACT_DTTM
0,id4651830,1954,2020-03-20 14:59:00,2020-12-01 00:00:00,255 days 09:01:00,1,2020-03-20 14:59:00
1,id5416547,1743,2020-03-17 10:17:00,2020-03-25 11:00:00,8 days 00:43:00,3,2020-03-17 10:17:00
2,id2185490,3210,2020-03-16 16:28:00,2020-12-01 00:00:00,259 days 07:32:00,2,2020-03-16 16:28:00
3,id7642700,3020,2020-03-15 14:21:00,2020-03-15 23:42:00,0 days 09:21:00,6,2020-03-15 14:21:00


In [106]:
success.ACT_DTTM.dt.strftime('%d-%m-%Y %H:%M')

0    20-03-2020 14:59
1    17-03-2020 10:17
2    16-03-2020 16:28
3    15-03-2020 14:21
Name: ACT_DTTM, dtype: object

In [109]:
interested_columns = ['ACT_DTTM', 'FILIAL_ID', 'difference']
success[interested_columns].to_csv('success_connections.csv', sep=';',index=False, date_format='%d-%m-%Y %H:%M')

In [107]:
??pd.DataFrame.to_csv

In [114]:
', '.join(success.SUBS_ID.sort_values().values)

'id2185490, id4651830, id5416547, id7642700'